In [26]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from groq import Groq

In [27]:
load_dotenv(override=True)
api_key = os.getenv('groud_cloud_key')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [30]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Scrap:
    def __init__(self,url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        
        

In [31]:
ed =Scrap("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [32]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [33]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [34]:
def messages_(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [48]:
def summarize(url):
    website = Scrap(url)
    client = Groq(api_key=api_key)
    response =  client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages = messages_(website),
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
        
    )
    full_output = "" 
    for chunk in response:
        content = chunk.choices[0].delta.content or ""
        full_output += content
    return full_output

#summarize("https://edwarddonner.com")

In [49]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://edwarddonner.com")

### Website Summary

The website of Edward Donner appears to be a personal professional portfolio and blog focused on AI, technology, and personal projects. The main topics covered include:

* **About the Author**: Edward Donner introduces himself as a tech enthusiast, coder, and experimenter with Large Language Models (LLMs). He shares his interests in writing code, DJing, electronic music production, and reading Hacker News.
* **Professional Background**: Ed is the co-founder and CTO of Nebula.io, where AI is applied to help people discover their potential. Previously, he founded and served as CEO of untapt, an AI startup acquired in 2021.

### Projects and Initiatives

* **Connect Four and Outsmart**: Two projects are highlighted:
  - **Connect Four**: No details provided.
  - **Outsmart**: An arena where LLMs compete against each other in diplomacy and strategy.

### News and Announcements

The website lists several upcoming or recent events and offerings:

* **May 28, 2025**: "Connecting my courses – become an LLM expert and leader"
* **May 18, 2025**: "2025 AI Executive Briefing"
* **April 21, 2025**: "The Complete Agentic AI Engineering Course"
* **January 23, 2025**: "LLM Workshop – Hands-on with Agents – resources"

These announcements suggest a focus on education and expertise in AI and LLMs, targeting both executives and technical professionals.

### Contact and Follow

The website provides several ways to connect with Edward Donner, including an email address, LinkedIn, Twitter, and Facebook profiles, as well as a subscription option for a newsletter.